In [1]:
from util import *
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
print("Data loading...")
city = 'bj'
df_aq = get_aq(city)
# df_meo = get_meo(city)
df_station = pd.read_csv("../input/" + city +"_aq_stations.csv")

print("Resampling...")
df_aq = df_resample(df_aq)
df_aq.head()

Data loading...
Resampling...


,utc_time,PM2.5,PM10,NO2,CO,O3,SO2,stationId
0,2017-01-01 22:00:00,453.0,467.0,156.0,7.2,3.0,9.0,aotizhongxin_aq
1,2017-01-01 23:00:00,417.0,443.0,143.0,6.8,2.0,8.0,aotizhongxin_aq
2,2017-01-02 00:00:00,395.0,467.0,141.0,6.9,3.0,8.0,aotizhongxin_aq
3,2017-01-02 01:00:00,420.0,484.0,139.0,7.4,3.0,9.0,aotizhongxin_aq
4,2017-01-02 02:00:00,453.0,520.0,157.0,7.6,4.0,9.0,aotizhongxin_aq


In [3]:
print("Calculating Top-k nearest stations...")
nears = ['near_1', 'near_2']
near_stations = cal_near_stations(df_station, nears)
df_station_near = pd.concat([df_station, pd.DataFrame(near_stations, columns=nears)], axis=1)

Calculating Top-k nearest stations...
0 dongsi_aq
0     0.000000
30    3.300267
1     3.370048
4     4.506715
dtype: float64 

1 tiantan_aq
1     0.000000
31    1.621681
30    1.643067
0     3.370048
dtype: float64 

2 guanyuan_aq
2     0.000000
32    2.157452
3     4.040461
33    6.288367
dtype: float64 

3 wanshouxigong_aq
3     0.000000
33    2.409265
2     4.040461
31    4.664349
dtype: float64 

4 aotizhongxin_aq
4     0.000000
0     4.506715
32    5.716710
30    6.146011
dtype: float64 

5 nongzhanguan_aq
5     0.000000
34    2.446481
0     4.919748
1     7.083157
dtype: float64 

6 wanliu_aq
6     0.000000
2     7.192861
32    7.302417
8     8.949106
dtype: float64 

7 beibuxinqu_aq
7      0.000000
8      7.471527
17    11.267700
11    13.071216
dtype: float64 

8 zhiwuyuan_aq
8     0.000000
11    6.994566
7     7.471527
6     8.949106
dtype: float64 

9 fengtaihuayuan_aq
9    0.000000
3    8.178674
2    8.258877
6    9.218868
dtype: float64 

10 yungang_aq
10    0.000000
12    

In [4]:
# Join station
try:
    del df_impute
except:
    pass

df_impute = pd.merge(df_aq, df_station_near, 'left', left_on='stationId', right_on='Station_ID')

# # Join meo
# df_impute.longitude = df_impute.longitude.round(1)
# df_impute.latitude = df_impute.latitude.round(1)
# df_impute = pd.merge(df_impute, df_meo, 'left', 
#               left_on=['utc_time', 'longitude', 'latitude'], 
#               right_on=['utc_time', 'longitude', 'latitude']
#              )

In [5]:
def spatial_step(df, target, nears):
    for near in nears:
        df = pd.merge(df, df[[target, 'stationId','utc_time']], 
                     how='left', suffixes=('', '_'+ near),
                     left_on=['utc_time', near], 
                     right_on=['utc_time', 'stationId'])
    return df

def temporal_step(df, target, city, EVALUATION=False):
    dfs_ = []
    nulls = []
    for station in df.stationId.unique():
        df_ = df[df.stationId==station]
        df_ = df_.sort_values('utc_time')

        for i in [-2, -1, 1, 2]:
            df_[target + "_t{0:+}".format(i)] = df_[target].shift(i*-1)

        cols = [target + "_t{0:+}".format(i) for i in [-2, -1, 1, 2]]
        cols += [target+'_near_1', target+'_near_2', target, 'utc_time']

        df_[target+"_pred"] = df_[cols].mean(axis=1) # stKNN: mean of 4 temperial + 2 spatial features

        # Update valie
        null_idxs = station_to_null_idxs[station]
        df_.loc[null_idxs, target] = df_[target + '_pred']
#         if EVALUATION:
#             eval_idxs = station_to_eval_idxs[station]
#             df_.loc[eval_idxs, target] = df_[target + '_pred']

        for i in [-2, -1, 1, 2]:
            df_[target + "_t{0:+}".format(i)] = df_[target].shift(i*-1)
        
        condition = df_[target].isnull() if city=="bj" else df_[target].isnull() & df_['need_prediction']==1
        null = df_[condition].shape[0]
        nulls.append(null)
#         print(N_null, station, target)
        dfs_.append(df_[['utc_time', 'stationId', target]])
    return dfs_, nulls



df_stKNNs = []
EVALUATION=False
station_to_null_idxs = {}
station_to_eval_idxs = {}
MSEs = []
    
target_cols = ['PM2.5', 'PM10', 'O3'] if city=='bj' else ['PM2.5', 'PM10']
# target_cols = ['humidity', 'pressure', 'temperature', 'wind_direction', 'wind_speed/kph']
# target_cols = ['temperature', 'wind_direction', 'wind_speed/kph']

for target in target_cols:
    print(target)
    df = df_impute.copy()
    
    for station in df.stationId.unique():
        df_ = df_impute[df.stationId==station]
        df_ = df_.sort_values('utc_time')
        if city=="bj":
            station_to_null_idxs[station] = df_[target].isnull() # for params update
#             if EVALUATION:
#                 eval_condition = (df_.utc_time>='2017-05-01 00:00:00') & (df_.utc_time<='2017-05-01 23:00:00') & (df_.stationId=="tiantan_aq") 
#                 station_to_eval_idxs[station] = eval_condition # for evaluation
        elif city=="ld":
            station_to_null_idxs[station] = (df_[target].isnull()) & (df_['need_prediction']==1) # for params update

    N_nulls = 999
    while N_nulls>0:
        df = spatial_step(df, target, nears)
        dfs, nulls = temporal_step(df, target, city)
        df = pd.concat(dfs)
        df = pd.merge(df, df_station_near, 'left', left_on='stationId', right_on='Station_ID')
        N_nulls = sum(nulls)
        print(N_nulls, nulls)
        
    df_stKNNs.append(df)

PM2.5
18724 [527, 532, 544, 538, 524, 527, 525, 558, 525, 529, 530, 524, 530, 563, 525, 524, 548, 553, 527, 524, 524, 528, 528, 529, 531, 524, 535, 528, 535, 525, 526, 551, 529, 533, 621]
16119 [458, 458, 462, 461, 457, 457, 458, 479, 457, 459, 460, 457, 461, 479, 457, 457, 468, 461, 457, 457, 457, 458, 460, 458, 460, 457, 461, 457, 463, 457, 457, 477, 461, 459, 457]
14931 [426, 425, 427, 425, 425, 425, 426, 435, 425, 427, 428, 425, 428, 435, 425, 425, 427, 425, 425, 425, 425, 425, 428, 426, 425, 425, 428, 425, 431, 425, 425, 432, 425, 427, 425]
14205 [406, 405, 407, 405, 405, 405, 406, 410, 405, 405, 405, 405, 407, 411, 405, 405, 407, 405, 405, 405, 405, 405, 405, 406, 405, 405, 407, 405, 407, 405, 405, 411, 405, 405, 405]
13711 [392, 391, 392, 391, 391, 391, 391, 395, 391, 391, 391, 391, 393, 397, 391, 391, 393, 391, 391, 391, 391, 391, 391, 391, 391, 391, 393, 391, 393, 391, 391, 397, 391, 391, 391]
13291 [380, 379, 380, 379, 379, 379, 379, 383, 379, 379, 379, 379, 381, 385, 379, 37

1901 [54, 54, 55, 54, 54, 54, 54, 54, 54, 54, 54, 54, 56, 56, 54, 54, 56, 54, 54, 54, 54, 54, 54, 54, 54, 54, 56, 54, 54, 54, 54, 56, 54, 54, 54]
1761 [50, 50, 51, 50, 50, 50, 50, 50, 50, 50, 50, 50, 52, 52, 50, 50, 52, 50, 50, 50, 50, 50, 50, 50, 50, 50, 52, 50, 50, 50, 50, 52, 50, 50, 50]
1621 [46, 46, 47, 46, 46, 46, 46, 46, 46, 46, 46, 46, 48, 48, 46, 46, 48, 46, 46, 46, 46, 46, 46, 46, 46, 46, 48, 46, 46, 46, 46, 48, 46, 46, 46]
1481 [42, 42, 43, 42, 42, 42, 42, 42, 42, 42, 42, 42, 44, 44, 42, 42, 44, 42, 42, 42, 42, 42, 42, 42, 42, 42, 44, 42, 42, 42, 42, 44, 42, 42, 42]
1341 [38, 38, 39, 38, 38, 38, 38, 38, 38, 38, 38, 38, 40, 40, 38, 38, 40, 38, 38, 38, 38, 38, 38, 38, 38, 38, 40, 38, 38, 38, 38, 40, 38, 38, 38]
1201 [34, 34, 35, 34, 34, 34, 34, 34, 34, 34, 34, 34, 36, 36, 34, 34, 36, 34, 34, 34, 34, 34, 34, 34, 34, 34, 36, 34, 34, 34, 34, 36, 34, 34, 34]
1061 [30, 30, 31, 30, 30, 30, 30, 30, 30, 30, 30, 30, 32, 32, 30, 30, 32, 30, 30, 30, 30, 30, 30, 30, 30, 30, 32, 30, 30, 30

4738 [134, 134, 135, 134, 136, 139, 134, 138, 134, 134, 134, 134, 137, 140, 134, 135, 139, 134, 134, 134, 137, 134, 136, 134, 134, 134, 136, 135, 136, 134, 137, 140, 134, 134, 136]
4458 [126, 126, 127, 126, 128, 131, 126, 130, 126, 126, 126, 126, 129, 132, 126, 127, 131, 126, 126, 126, 129, 126, 128, 126, 126, 126, 128, 127, 128, 126, 129, 132, 126, 126, 128]
4178 [118, 118, 119, 118, 120, 123, 118, 122, 118, 118, 118, 118, 121, 124, 118, 119, 123, 118, 118, 118, 121, 118, 120, 118, 118, 118, 120, 119, 120, 118, 121, 124, 118, 118, 120]
3898 [110, 110, 111, 110, 112, 115, 110, 114, 110, 110, 110, 110, 113, 116, 110, 111, 115, 110, 110, 110, 113, 110, 112, 110, 110, 110, 112, 111, 112, 110, 113, 116, 110, 110, 112]
3618 [102, 102, 103, 102, 104, 107, 102, 106, 102, 102, 102, 102, 105, 108, 102, 103, 107, 102, 102, 102, 105, 102, 104, 102, 102, 102, 104, 103, 104, 102, 105, 108, 102, 102, 104]
3338 [94, 94, 95, 94, 96, 99, 94, 98, 94, 94, 94, 94, 97, 100, 94, 95, 99, 94, 94, 94, 97, 94, 

7785 [222, 222, 223, 222, 222, 222, 222, 226, 222, 222, 222, 222, 222, 228, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 224, 222, 222, 224, 222, 222, 222]
7505 [214, 214, 215, 214, 214, 214, 214, 218, 214, 214, 214, 214, 214, 220, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 216, 214, 214, 216, 214, 214, 214]
7225 [206, 206, 207, 206, 206, 206, 206, 210, 206, 206, 206, 206, 206, 212, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 208, 206, 206, 208, 206, 206, 206]
6945 [198, 198, 199, 198, 198, 198, 198, 202, 198, 198, 198, 198, 198, 204, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 200, 198, 198, 200, 198, 198, 198]
6665 [190, 190, 191, 190, 190, 190, 190, 194, 190, 190, 190, 190, 190, 196, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 192, 190, 190, 192, 190, 190, 190]
6385 [182, 182, 183, 182, 182, 182, 182, 186, 182, 182, 182, 182, 182, 188, 182, 182, 182, 182,

## Merge & 检查是否正确

In [6]:
df = df_impute.copy()

for i, target in enumerate(target_cols):
    df = pd.merge(df, df_stKNNs[i][['utc_time', 'stationId', target]], how='left', suffixes=("", "_pred"),
                  left_on=['utc_time', 'stationId'],  right_on=['utc_time', 'stationId'],
                 )
    
    tmp = df[pd.notnull(df[target])]
    MAE = mean_absolute_error(tmp[target], tmp[target+'_pred'])
    print(target, MAE)
    
    df[target] = df[target + '_pred']
    
df = df[['utc_time'] + target_cols + ['stationId']]
df.to_csv("../input/" + city + "_aq_imputed.csv", index=False)


PM2.5 0.0
PM10 0.0
O3 0.0


## Evalution (手动选择某一站点，设置一天空值)

In [ ]:
# condition = (df.stationId=="tiantan_aq") & (df.utc_time>='2017-05-01 00:00:00') & (df.utc_time<='2017-05-01 23:00:00')
# tmp = df[['utc_time','PM10', 'PM10_pred']][condition]
# tmp = tmp.sort_values(by='utc_time')
# # tmp
# tmp.plot(x='utc_time', y=['PM10', 'PM10_pred'], figsize=(16,6), style={'PM10': '-', 'PM10_pred': ':'})

# tmp = df[condition]
# tmp = tmp[pd.notnull(tmp[target])]
# MAE = mean_absolute_error(tmp[target], tmp[target+'_pred'])
# print(MAE)

In [ ]:
tmp = df_impute[df_impute.stationId=="aotizhongxin_aq"]
tmp = tmp.sort_values('utc_time')
tmp[target].isnull() # for params update
station_to_null_idxs['aotizhongxin_aq'].shape
df_impute.dtypes
df_impute